## Objectif : obtenir une carte qui représente les ports navigo et bureaux des Fermes Toflit concernés par le sprint
- légende particulière quand un port est un bureau des Fermes
- colorer de la même manière tous les ports rattachés à un même bureau des Fermes

=> Kepler et / ou Khartis

In [1]:
from poitousprint import Portic, Toflit, get_pointcalls_commodity_purposes_as_toflit_product

toflit_client = Toflit()
portic_client = Portic()

In [6]:
# récupération de tous les ports navigo concernés

pointcalls_datasprint = portic_client.get_pointcalls(
 year = 1789,
 pointcall_admiralty = ['La Rochelle', 'Marennes', 'Sables d\'Olonne']
    # avec source_subset on otbient des ports hors de la région (car ils ont des flux en relation avec ceux de la région)
)

ports_navigo_dict = {} 
for pointcall in pointcalls_datasprint:
    is_bureau = False

    # si le port n'est pas encore enregistré dans les valeurs existantes on initialise les valeurs associées
    if pointcall['pointcall'] not in ports_navigo_dict.keys():
        if pointcall['pointcall'] == pointcall['ferme_bureau'] or pointcall['pointcall'] == 'Aligre de Marans' or pointcall['pointcall'] == 'Saint Martin de Ré': # ne marche pas pour certains noms ... (--> 'Aligre' ≠ 'Aligre de Marrans')
            is_bureau = True 
        
        # remplacer par "pas de bureau rattaché" plutot que None quand le port Navigo n'est pas rattaché à un bureau des Fermes Toflit18
        bureau = pointcall['ferme_bureau']
        if bureau is None:
            bureau = "pas de bureau rattaché"
            
        ports_navigo_dict[pointcall['pointcall']] = {
            'latitude':pointcall['latitude'],
            'longitude':pointcall['longitude'],
            'bureau':bureau,
            'is_bureau':is_bureau,
            'pointcall_count':1
            }
    
    #  si le port est déjà enregistré dans les valeurs existantes on incrémente seulement le compteur du nombre de pointcalls 
    else:
        ports_navigo_dict[pointcall['pointcall']]['pointcall_count'] += 1

ports_navigo_dict

{'Ile de Ré': {'latitude': '46.2',
  'longitude': '-1.416667',
  'bureau': 'pas de bureau rattaché',
  'is_bureau': False,
  'pointcall_count': 200},
 'Esnandes': {'latitude': '46.25',
  'longitude': '-1.083333',
  'bureau': 'La Rochelle',
  'is_bureau': False,
  'pointcall_count': 39},
 'Le Plomb': {'latitude': '46.2',
  'longitude': '-1.2',
  'bureau': 'pas de bureau rattaché',
  'is_bureau': False,
  'pointcall_count': 12},
 'Oléron': {'latitude': '45.933333',
  'longitude': '-1.3',
  'bureau': 'pas de bureau rattaché',
  'is_bureau': False,
  'pointcall_count': 405},
 'Marennes': {'latitude': '45.816667',
  'longitude': '-1.116667',
  'bureau': 'Marennes',
  'is_bureau': True,
  'pointcall_count': 1709},
 'La Rochelle': {'latitude': '46.166667',
  'longitude': '-1.15',
  'bureau': 'La Rochelle',
  'is_bureau': True,
  'pointcall_count': 1950},
 'Rochefort': {'latitude': '45.943666',
  'longitude': '-0.97043',
  'bureau': 'Rochefort',
  'is_bureau': True,
  'pointcall_count': 572},


In [4]:
# Vérification : récupération des bureaux des Fermes concernés chez Toflit
flows = toflit_client.get_flows(
    year=1789, 
    customs_region='La Rochelle',
)

toflit_customs_offices_for_datasprint = set()
for flow in flows:
    toflit_customs_offices_for_datasprint.add(flow['customs_office'])

toflit_customs_offices_for_datasprint

{'',
 'Aligre',
 'Charente',
 'La Rochelle',
 "Les Sables d'Olonne",
 'Marennes',
 'Rochefort',
 'Saint-Martin-de-Ré'}

In [7]:
# écriture dans un csv
import csv
from csv import DictWriter

# tableau qui contient une colonne par clé de dict 
with open('dumps/croisement_ports_bureaux_datasprint_aggregated.csv', 'w', newline='') as csvfile:
        fieldnames = ['pointcall','latitude', 'longitude', 'bureau', 'is_bureau', 'pointcall_count']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for port, values in ports_navigo_dict.items():
                    
            writer.writerow({
                'pointcall': port,
                'latitude': values['latitude'],
                'longitude': values['longitude'],
                'bureau': values['bureau'],
                'is_bureau': values['is_bureau'],
                'pointcall_count': values['pointcall_count']
            })

In [8]:
# Load an empty map
from keplergl import KeplerGl
from poitousprint import Portic
import pandas as pd

portic_client = Portic()

In [20]:
# setting kepler config
config = {
    'version': 'v1',
    'config': {
        # centering the map on the region
        'mapState': {
            'latitude': 45.6876849,
            'longitude': -1.15,
            'zoom': 5.2
        }
    }
}

df = pd.read_csv('dumps/croisement_ports_bureaux_datasprint_aggregated.csv')
# reste à faire les réglages de couleur, proportionalité, mise en avant des bureaux ...

flux_map = KeplerGl(config=config, height=800, data={'data_1': df})
flux_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'mapState': {'latitude': 45.6876849, 'longitude': -1.15, 'zoom': …

## Config intéressante

In [18]:
config = {
  "version": "v1",
  "config": {
    "visState": {
      "filters": [],
      "layers": [
        {
          "id": "5fl7ry",
          "type": "point",
          "config": {
            "dataId": "data_1",
            "label": "Point",
            "color": [
              255,
              203,
              153
            ],
            "columns": {
              "lat": "latitude",
              "lng": "longitude",
              "altitude": None
            },
            "isVisible": True,
            "visConfig": {
              "radius": 16.7,
              "fixedRadius": False,
              "opacity": 0.8,
              "outline": True,
              "thickness": 2,
              "strokeColor": None,
              "colorRange": {
                "name": "Custom Palette",
                "type": "custom",
                "category": "Custom",
                "colors": [
                  "#e41a1c",
                  "#377eb8",
                  "#4daf4a",
                  "#984ea3",
                  "#ff7f00",
                  "#ffff33",
                  "#a65628"
                ]
              },
              "strokeColorRange": {
                "name": "Custom Palette",
                "type": "custom",
                "category": "Custom",
                "colors": [
                  "#000000",
                  "#FFFFFF"
                ]
              },
              "radiusRange": [
                12.7,
                50
              ],
              "filled": True
            },
            "hidden": False,
            "textLabel": []
          },
          "visualChannels": {
            "colorField": {
              "name": "bureau",
              "type": "string"
            },
            "colorScale": "ordinal",
            "strokeColorField": {
              "name": "is_bureau",
              "type": "boolean"
            },
            "strokeColorScale": "ordinal",
            "sizeField": {
              "name": "pointcall_count",
              "type": "integer"
            },
            "sizeScale": "sqrt"
          }
        }
      ],
      "interactionConfig": {
        "tooltip": {
          "fieldsToShow": {
            "data_1": [
              {
                "name": "pointcall",
                "format": None
              },
              {
                "name": "bureau",
                "format": None
              },
              {
                "name": "is_bureau",
                "format": None
              },
              {
                "name": "pointcall_count",
                "format": None
              }
            ]
          },
          "compareMode": False,
          "compareType": "absolute",
          "enabled": True
        },
        "brush": {
          "size": 9.4,
          "enabled": False
        },
        "geocoder": {
          "enabled": False
        },
        "coordinate": {
          "enabled": False
        }
      },
      "layerBlending": "normal",
      "splitMaps": [],
      "animationConfig": {
        "currentTime": None,
        "speed": 1
      }
    },
    "mapState": {
      "bearing": 0,
      "dragRotate": False,
      "latitude": 45.94994663229237,
      "longitude": -0.9635546220911514,
      "pitch": 0,
      "zoom": 8.5, #7.469414939418524,
      "isSplit": False
    },
    "mapStyle": {
      "styleType": "dark",
      "topLayerGroups": {},
      "visibleLayerGroups": {
        "label": True,
        "road": True,
        "border": False,
        "building": True,
        "water": True,
        "land": True,
        "3d building": False
      },
      "threeDBuildingColor": [
        9.665468314072013,
        17.18305478057247,
        31.1442867897876
      ],
      "mapStyles": {}
    }
  }
}

flux_map = KeplerGl(config=config, height=800, data={'data_1': df})
flux_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': '5fl7ry', 'type': '…